# SECOM Dataset - EDA

Quick look at the dataset before building the model. Main things to figure out:
- How imbalanced are the classes?
- How bad is the missing data?
- Which features are mostly empty and should be dropped?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/uci-secom.csv')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
df.dtypes.value_counts()

## Class distribution

Labels are -1 (pass) and 1 (fail). Expecting heavy imbalance.

In [ ]:
label_counts = df['Pass/Fail'].value_counts()
print(label_counts)
print(f'\nImbalance ratio: {label_counts[-1] / label_counts[1]:.1f}:1 (pass:fail)')

label_counts.plot(kind='bar', color=['steelblue', 'salmon'])
plt.title('Pass vs Fail')
plt.xticks([0, 1], ['Pass (-1)', 'Fail (1)'], rotation=0)
plt.ylabel('Count')
plt.show()

## Missing values

SECOM is known for having tons of NaNs. Need to figure out which columns are mostly empty so we can drop them.

In [ ]:
# drop non-feature columns for this analysis
features = df.drop(columns=['Time', 'Pass/Fail'])

missing_pct = (features.isnull().sum() / len(features)) * 100
print(f'Total features: {len(missing_pct)}')
print(f'Features with any missing: {(missing_pct > 0).sum()}')
print(f'Features with >50% missing: {(missing_pct > 50).sum()}')
print(f'Features with zero missing: {(missing_pct == 0).sum()}')

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(missing_pct, bins=50, edgecolor='black')
plt.xlabel('% Missing')
plt.ylabel('Number of Features')
plt.title('Missing Value Distribution Across Features')
plt.axvline(x=50, color='red', linestyle='--', label='50% threshold')
plt.legend()
plt.tight_layout()
plt.show()

# worst offenders
missing_pct.sort_values(ascending=False).head(15)

In [ ]:
## Preprocessing Demo
Showing the cleaning, scaling, and split that happens in the training pipeline.

In [ ]:
# drop features with >50% missing
threshold = 50
cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
df_clean = df.drop(columns=['Time'] + cols_to_drop)
print(f"Dropped {len(cols_to_drop)} columns with >{threshold}% missing")
print(f"Remaining features: {df_clean.shape[1] - 1}")  # minus the label column

In [ ]:
# impute remaining missing values with median
from sklearn.impute import SimpleImputer

X = df_clean.drop(columns=['Pass/Fail']).values
y = df_clean['Pass/Fail'].values

# convert labels from -1/1 to 0/1
y = np.where(y == -1, 0, y)

imputer = SimpleImputer(strategy='median')
X = imputer.fit_transform(X)
print(f"Missing values after imputation: {np.isnan(X).sum()}")

In [ ]:
# impute remaining missing values with median
from sklearn.impute import SimpleImputer

X = df_clean.drop(columns=['Pass/Fail']).values
y = df_clean['Pass/Fail'].values

# convert labels from -1/1 to 0/1
y = np.where(y == -1, 0, y)

imputer = SimpleImputer(strategy='median')
X = imputer.fit_transform(X)
print(f"Missing values after imputation: {np.isnan(X).sum()}")

In [ ]:
# standardize features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Before scaling - mean: {X.mean():.2f}, std: {X.std():.2f}")
print(f"After scaling  - mean: {X_scaled.mean():.4f}, std: {X_scaled.std():.4f}")

In [ ]:
# stratified train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train set: {X_train.shape[0]} samples, defect rate: {y_train.mean():.3f}")
print(f"Test set:  {X_test.shape[0]} samples, defect rate: {y_test.mean():.3f}")
print(f"\nBoth ~{y.mean():.3f} - stratification preserved the ratio")